In [1]:
import os
import random
import numpy as np
from skimage import io
from skimage.color import rgba2rgb

# PATHS
input_root = "../imgs_with_good_background"
output_root = "./img_outs"
# valid file extensions tuple
file_exts = (".jpg", ".jpeg", ".png")
# varying background folder dirs, for the bacterial blight class
bkgd_dirs_bact_blight = ["bkgd1", "bkgd2", "bkgd_none"]

# Lighting simulation parameters 
gen_seed = 34352
lower_frac = 0.5
upper_frac = 1.5
switch_frac_threshold = 1.0
min_ambient_frac = 0.03

# Normalized RGB values derived from the following sites:
# https://academo.org/demos/colour-temperature-relationship/
# https://www.softlights.org/chapter-11-color-temperature/
sun_horz_lighting_color_vec = np.array([1.0, 0.81, 0.65]).reshape(1, 1, 3)
noon_lighting_color_vec = np.array([1.0, 0.96, 0.93]).reshape(1, 1, 3)

random.seed(gen_seed)

def apply_lighting_effect(img, orig_input_path):
    
    if img.ndim == 2:  # Grayscale
        print(f"Skipping grayscale image: {orig_input_path}")
        return None
    elif img.shape[-1] == 4:  # RGBA to RGB
        img = rgba2rgb(img)

    # Normalize
    img_raw_normed = img.astype(np.float64) / 255.0

    # Lighting modification
    selected_intensity_frac = random.uniform(lower_frac, upper_frac)
    res_img_raw = None
    # Inspired by https://github.com/ayulockin/PaintingLight/blob/master/code/ProjectPaintingLight.py#L216
    if selected_intensity_frac >= switch_frac_threshold:
        res_img_raw = (selected_intensity_frac * noon_lighting_color_vec * img_raw_normed
                        + min_ambient_frac * noon_lighting_color_vec)
    else:
        res_img_raw = selected_intensity_frac * sun_horz_lighting_color_vec * img_raw_normed

    # Final image conversion
    fin_procd_img = np.round(np.clip(res_img_raw, 0.0, 1.0) * 255.0).astype(np.uint8)

    return fin_procd_img                


proc_count = 0

# Traverse and process all images 
for label in os.listdir(input_root):
    class_folder = os.path.join(input_root, label)
    if not os.path.isdir(class_folder):
        continue

    # Create output subfolder
    output_class_folder = os.path.join(output_root, label)
    os.makedirs(output_class_folder, exist_ok=True)

    # Process each image in the class folder
    for filename in os.listdir(class_folder):
              
        if any([filename == dir_name \
                    for dir_name in bkgd_dirs_bact_blight]):
            subdir_path = os.path.join(class_folder, filename)
            output_subclass_folder = os.path.join(output_class_folder, filename)
            os.makedirs(output_subclass_folder, exist_ok=True)
            for sub_filename in os.listdir(subdir_path):
                if sub_filename.lower().endswith(file_exts):
                    img_path = os.path.join(subdir_path, sub_filename)
                    try:
                        img = io.imread(img_path)
                        fin_procd_img = apply_lighting_effect(img, img_path)
                        rel_path = os.path.relpath(img_path, subdir_path)
                        rel_path_png = os.path.splitext(rel_path)[0] + ".png"
                        output_path = os.path.join(output_subclass_folder, rel_path_png)
                        io.imsave(output_path, fin_procd_img)
                        proc_count += 1
                        if proc_count % 25 == 0:
                            print(f"Processed {proc_count} images...")
                    except Exception as e:
                        print(f"❌ Error processing {img_path}: {e}")
                
        elif filename.lower().endswith(file_exts):
            img_path = os.path.join(class_folder, filename)
            try:
                img = io.imread(img_path)
                fin_procd_img = apply_lighting_effect(img, img_path)
                rel_path = os.path.relpath(img_path, class_folder)
                rel_path_png = os.path.splitext(rel_path)[0] + ".png"
                output_path = os.path.join(output_class_folder, rel_path_png)
                io.imsave(output_path, fin_procd_img)
                proc_count += 1
                if proc_count % 25 == 0:
                    print(f"Processed {proc_count} images...")
            except Exception as e:
                print(f"❌ Error processing {img_path}: {e}")

print("✅ All images processed and saved with lighting variations.")

Processed 25 images...
Processed 50 images...
Processed 75 images...
Processed 100 images...
Processed 125 images...
Processed 150 images...
Processed 175 images...
Processed 200 images...
Processed 225 images...
Processed 250 images...
Processed 275 images...
Processed 300 images...
Processed 325 images...
Processed 350 images...
Processed 375 images...
Processed 400 images...
✅ All images processed and saved with lighting variations.
